# Solution

In [24]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
import PIL
import os
import sys
import glob
import random

%load_ext autoreload
%autoreload 2
import utilities as utils

from pprint import pprint
from ipywidgets import Video

from PIL import Image as PILImage
from PIL.ExifTags import TAGS

from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create solution video

In [26]:
special_tiles = [
        (4, 0),
        (0, 1),
        (5, 2),
        (9, 2),
        (3, 4),
        (6, 4),
        (8, 6),
        (3, 7),
        (0, 8),
        (5, 9)
    ]

In [29]:
inputs = []

for filename in inputs:
    print(f"Processing {filename}")

    input_file = f"Videos/{filename}"
    dice_template = cv2.imread("Images/dice_template.jpg", cv2.IMREAD_GRAYSCALE)

    cap = cv2.VideoCapture(input_file)

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    output_file = f"Solutions/Final/final_{filename}"
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width + 570, frame_height))

    out_width = frame_width + 570
    out_high = frame_height

    ret, frame = cap.read()
    M, warped_width, warped_height = utils.detect_board(frame)
    board = cv2.warpPerspective(frame, M, (int(warped_width), int(warped_height)))

    M_inv = np.linalg.inv(M)

    count = 0
    state = utils.get_state(board)
    
    green, red, yellow, blue = state

    green_max = green["score"] if green["score"] is not None else 0
    red_max = red["score"] if red["score"] is not None else 0
    yellow_max = yellow["score"] if yellow["score"] is not None else 0
    blue_max = blue["score"] if blue["score"] is not None else 0

    got_coin = {"Green": -99999, "Red": -99999, "Yellow": -99999, "Blue": -99999} 

    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    dice_center, dice_top_left, dice_bottom_right = utils.detect_dice(frame, dice_template)
    number_of_pips = utils.count_dice_pips(frame, dice_center)
    p0 = np.array([[dice_center]], dtype=np.float32)
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if count % 10 == 0:
            board = cv2.warpPerspective(frame, M, (int(warped_width), int(warped_height)))

            state = utils.get_state(board, state)
            green, red, yellow, blue = state
            
            if green["score"] is not None and green["score"] > green_max and green["coord"] in special_tiles:
                green_max = green["score"]
                got_coin["Green"] = count
        
            if red["score"] is not None and red["score"] > red_max and red["coord"] in special_tiles:
                red_max = red["score"]
                got_coin["Red"] = count

            if yellow["score"] is not None and yellow["score"] > yellow_max and yellow["coord"] in special_tiles:
                yellow_max = yellow["score"]
                got_coin["Yellow"] = count

            if blue["score"] is not None and blue["score"] > blue_max and blue["coord"] in special_tiles:
                blue_max = blue["score"]
                got_coin["Blue"] = count

        if count % 60 == 0:
            dice_center, top_left, bottom_right = utils.detect_dice(frame, dice_template)
            p0 = np.array([[dice_center]], dtype=np.float32)

        board_copy = board.copy()    
        utils.annotate_board(board_copy, state)
        prev_frame, p0, dice_center, dice_top_left, dice_bottom_right, number_of_pips, is_rolling = utils.annotate_frame(frame,
                                                                                                                         prev_frame,
                                                                                                                         dice_template,
                                                                                                                         lk_params,
                                                                                                                         p0,
                                                                                                                         dice_center,
                                                                                                                         dice_top_left,
                                                                                                                         dice_bottom_right,
                                                                                                                         number_of_pips,
                                                                                                                         board_copy,
                                                                                                                         state,
                                                                                                                         M_inv)

        canvas = np.ones((frame.shape[0], frame.shape[1] + 570, 3), dtype=np.uint8) * 255
        canvas[:frame.shape[0], :frame.shape[1]] = frame

        board_copy = cv2.resize(board_copy, (board_copy.shape[1]//5, board_copy.shape[0]//5))
        start_x = canvas.shape[1] - board_copy.shape[1] - 40
        start_y = canvas.shape[0] - board_copy.shape[0] 
        canvas[start_y: start_y+board_copy.shape[0], start_x:start_x+board_copy.shape[1]] = board_copy

        utils.annotate_state(canvas, frame, state)

        utils.annotate_events(canvas, frame, state, number_of_pips, is_rolling)

        utils.annotate_score_change(canvas, frame, got_coin, count)

        out.write(canvas)

        count+=1

        if count%300 == 0:
            print(f"Processed {count} frames out of {frame_count}")

    print("DONE")
    print()

    cap.release()
    out.release()

Processing easy_1.mp4
Processed 300 frames out of 2046
Processed 600 frames out of 2046
Processed 900 frames out of 2046
Processed 1200 frames out of 2046
Processed 1500 frames out of 2046
Processed 1800 frames out of 2046
DONE

Processing easy_2.mp4
Processed 300 frames out of 2173
Processed 600 frames out of 2173
Processed 900 frames out of 2173
Processed 1200 frames out of 2173
Processed 1500 frames out of 2173
Processed 1800 frames out of 2173
Processed 2100 frames out of 2173
DONE

Processing easy_3.mp4
Processed 300 frames out of 3005
Processed 600 frames out of 3005
Processed 900 frames out of 3005
Processed 1200 frames out of 3005
Processed 1500 frames out of 3005
Processed 1800 frames out of 3005
Processed 2100 frames out of 3005
Processed 2400 frames out of 3005
Processed 2700 frames out of 3005
Processed 3000 frames out of 3005
DONE

